# Python - R Crosstalk (cont.)
## Jupyter Magic
Jupyter R magic can be used to make calls to R from Jupyter Notebooks and to return outputs from R functions into the Python environment.

In [ ]:
%load_ext rpy2.ipython

In [ ]:
%%R -o x
x = seq(5)

In [ ]:
x

## Pandas Follow-ups
Pandas can read tabulated data directly from URLs. 

In [ ]:
import numpy as np
from pandas import read_csv

## Read data from URL.
url = 'https://raw.githubusercontent.com/szorowi1/qmss2017/master/module3/random/random.csv'
data = read_csv(url)
data.head(3)

Variables can be set as new Pandas indices using the **set_index** attribute.

In [ ]:
data = data.set_index('y')
data.head(3)

This can be especially useful for fast indexing of specific outcomes. 

In [ ]:
data.loc['hotel'].head(5)

## dplyr in Python: *dplython* library
To install, open the Terminal and type:
```
pip install dplython
```
For full documentation, please see [here](https://github.com/dodger487/dplython).

In [ ]:
from dplython import DplyFrame, X, select, sift, head, mutate, group_by, summarize

## Convert DataFrame to DplyFrame
data_dply = DplyFrame(data.reset_index())

## Example with sift, select, and head.
data_dply >> sift( X.x01 < -5 ) >> select( X.x02, X.x03, X.x04 ) >> head(5)

In [ ]:
## Example with mutate, group_by, and summarize.
(data_dply >> 
  mutate(x01_bin=X.x01.round()) >> 
  group_by(X.y, X.x01_bin) >> 
  summarize(x02_mean=X.x02.mean())) >> head(10)

## dplyr in Python: *pandas-ply* library
To install, open the Terminal and type:
```
pip install pandas-ply
```
For full documentation, please see [here](https://pythonhosted.org/pandas-ply/).

In [ ]:
import pandas as pd
from pandas_ply import install_ply, X, sym_call

## After calling install_ply, all pandas objects have 
## pandas-ply‘s methods attached.
install_ply(pd)

## Piping example.
(data
 .reset_index()
 .groupby('y')
 .ply_select(
    arr = X.x01.mean(),
    dep = X.x02.mean())
 .ply_where(X.arr < -5, X.dep > 7))

## dplyr in Python: *rpy2* library

For more information, please see [here](http://blog.yhat.com/posts/rpy2-combing-the-power-of-r-and-python.html). 

To import functions from R, use **robjects.**

In [2]:
import rpy2.robjects as robjects

seq = robjects.r('seq')
seq(5)

R object with classes: ('integer',) mapped to:
<IntVector - Python:0x10b478ac8 / R:0x7f9b6bb9d898>
[       1,        2,        3,        4,        5]

To import whole libraries, use **importr**. 

In [5]:
from rpy2.robjects.packages import importr

lme4 = importr('lme4')
lme4?

# Scikit-Learn Tutorial

The following will be a brief overview of machine learning with python. For more complete treatments, please see:
* [Introduction to Machine Learning with Python](http://shop.oreilly.com/product/0636920030515.do)

## Overview
Scikit-Learn, or sklearn, is the foremost machine learning library in Python. It contains a number of state-of-the-art machine learning algorithms, as well as comprehensive [documentation](http://scikit-learn.org/stable/index.html). It is impressively fast, and the functions are constructed in such a way so as to involve the least amount of code possible for the user. Moreover, Scikit-Learn is designed for maximal compatibility with Pandas, making the training and testing of data very easy. 

At the highest level of description, all algorithms in scikit-learn are implemented as pythonic classes. All algorithms must first be initialzied (i.e. must initialize an instance of the class) with specified parameters. All algorithmic intances have a "fit" attribute for applying the algorithm to data. NumPy arrays, sparse data, and Pandas data structures are all accepted as inputs. 

To practice with Scikit-Learn, we will be applying some machine learning techniques to both "clean" and "messy" datasets. The purpose for using multiple datasets is to highlight the validity of the code with clean data (by reproducing classic clustering/classification/regression results), and demonstrate more complex use cases with noisier data. We introduce the datasets below.

## Data Coding

In [6]:
data.describe().round(3)

NameError: name 'data' is not defined

### Dummy Coding

In [ ]:
from pandas import get_dummies

## Make dummy coded variables.
get_dummies(data.index).head(10)

### Digitizing

In [ ]:
## Define bins.
bins = [-9,-7,-5,-3,-1]

x = data.x01
xd = np.digitize(x, bins)

for a,b in np.c_[x,xd][::20]:
    print('%0.3f\t%0.0f' %(a,b))

### Data Scaling
Scikit-Learn includes several built-in preprocessing algorithms for transforming/recoding data:
* **StandardScaler:** scales a variable such that it is mean = 0, sd = 1
* **RobustScaler:** scales a variable using medians and quartiles (i.e. 25% of data falls beneath 1st quartile)
* **MinMaxScaler:** scales a variable between a specified min and max (e.g. [0,1])
* **Normalizer:** scales each data point such that the feature vector has a euclidean length of one

We will test two of these now, applying the RobustScaler to all continous data and the MinMaxScaler to all nominal data.

In [ ]:
from sklearn.preprocessing import StandardScaler

## Apply standard-score (z-score) transform to data. 
data.loc[:,:] = StandardScaler().fit_transform(data.loc[:,:])

data.describe().round(3)

## Feature Selection & Dimensionality Reduction

### Variance Inflation Factor

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style('darkgrid')
sns.set_context('notebook', font_scale=2)
%matplotlib inline

sns.heatmap(data.corr(), vmin=-1, vmax=1, square=True);

In [ ]:
from statsmodels.stats.outliers_influence import variance_inflation_factor
variance_inflation_factor?

In [ ]:
## Compute VIF.
VIF = [variance_inflation_factor(data.as_matrix(), n) for n in range(data.shape[-1])]

## Plot.
fig = plt.figure(figsize=(12,4))
color, = sns.color_palette(n_colors=1)
ax = sns.barplot(np.arange(data.shape[-1]), VIF, color=color);
ax.set(xticklabels=data.columns,  ylabel='VIF');

In [ ]:
## Drop column x09.
data = data.drop('x09', axis=1)

## Compute VIF.
VIF = [variance_inflation_factor(data.as_matrix(), n) for n in range(data.shape[-1])]

## Plot.
fig = plt.figure(figsize=(12,4))
color, = sns.color_palette(n_colors=1)
ax = sns.barplot(np.arange(data.shape[-1]), VIF, color=color);
ax.set(xticklabels=data.columns, ylim=(0,15), ylabel='VIF');

### Feature Selection
Scikit-Learn also includes a number of featuers for the automatic reduction of features to those most explanatorily useful. These include:
* **Univariate approches:** selection of features with statistically significant relationships to the target. These approaches only consider each feature individually. Consequently a feature will be discarded if it is only informative when combined with another feature. 
* **Model-based approaches:** use a supervised machine learning model to judge the importance of each feature, and keeps only the most important ones. The supervised model that is used for feature selection doesn’t need to be the same model that is used for the final supervised modeling.
* **Iterative selection:** a series of models is built, with varying numbers of features. There are two basic methods: starting with no features and adding features one by one, until some stopping criterion is reached, or starting with all features and removing features one by one, until some stopping criterion is reached. 

Model-based and iterative selection methods are more complicated, powerful approaches. For now, we will demonstrate feature selection with the simpler univariate approaches. There are several functions to choose from including **SelectKBest**, **SelectPercentile**, **SelectFdr**, and **SelectFwe**. As the names imply, the first two functions select the K-best and N%-best features, whereas the latter two select features that are statistically significant after multiple comparisons corrections.

With 271 observations and 159 features, the phenotype dataset could stand to use a little more pruning.

In [ ]:
from sklearn.feature_selection import SelectPercentile, SelectFdr, SelectFwe

## Initialize feature selection classes.
fwe = SelectFwe(alpha=0.05)

## Fit to data.
fit = fwe.fit(data, data.index)

## Find surviving columns.
fwe_cols = fit.get_support()
fwe_cols

Scikit-Learn attributes can be chained.

In [ ]:
fwe_cols = SelectFwe(alpha=0.05).fit(data,data.index).get_support()
fwe_cols

Reduce data to surviving columns.

In [ ]:
data = data[data.columns[fwe_cols]]
data.head(5)

## Unsupervised Learning

In [ ]:
from sklearn.decomposition import PCA

## Initialize with desired number of components.
pca = PCA(n_components=2)

## Fit to data.
fit = pca.fit(data)
print(fit.explained_variance_ratio_)

## Transform.
data_2d = fit.transform(data)
print(data_2d.shape)

In [ ]:
from pandas import DataFrame

## Fit into DataFrame.
data_2d = DataFrame(fit.transform(data), columns=('PCA1','PCA2'), index=data.index)

## Plot.
y_types = data_2d.index.unique()
colors = sns.color_palette(n_colors=len(y_types))

fig, ax = plt.subplots(1,1,figsize=(8,6))
for y, color in zip(y_types, colors):
    ax.scatter(data_2d.loc[y, 'PCA1'], data_2d.loc[y, 'PCA2'], color=color, label=y)
ax.set(xlabel='PCA1', ylabel='PCA2')
ax.legend(loc=7, bbox_to_anchor=(1.25,0.5), handletextpad=0.1);

## Supervised Learning

### Train and Test Sets
Scikit-learn contains a function, **train_test_split**, that shuffles the dataset and splits it for you. This is perhaps the most essential function in all of Scikit-Learn. Fortunately it handles Pandas arrays making it easily compatible with any data that can be read into Python via Pandas. We demonstrate this function below on the diabetes dataset.

In [ ]:
from sklearn.model_selection import train_test_split

## Separate out precictors and outcomes.
X = data.as_matrix()
y = data.index

## Split our dataset into four variables using train_test_split.
## We hold out 20% of the data for prediction. All but the last
## column are predictors.
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.80, test_size=0.2, random_state=0)

print('Original data shape: (%s, %s)' %data.shape)
print('Train data shape: (%s, %s)' %X_train.shape)
print('Test data shape: (%s, %s)' %X_test.shape)

### Fitting Models

In [ ]:
from sklearn.svm import LinearSVC

## Initialize SVC with desired parameteres.
svc = LinearSVC(penalty='l2', C=1.0, random_state=0)

## Fit to data.
fit = svc.fit(X_train, y_train)

## Compute predictive accuracy to test data.
print('Training performance: %0.3f' %fit.score(X_train, y_train))
print('Test performance: %0.3f' %fit.score(X_test, y_test))

## Show predictions.
predictions = fit.predict(X_test)
predictions

In [ ]:
from sklearn.metrics import confusion_matrix

confmat = DataFrame(confusion_matrix(y_test, predictions), columns=np.unique(y_test), 
                    index=np.unique(y_test))
confmat = confmat.apply(lambda x: x / x.sum(), 1)

confmat.round(3)

### Cross-Validation
One downside to train_test_split is that it splits the data only once. This can be problematic as that one model fit may be unrepresentative of the underlying true model fit as a result of the random allocation of data to the train/test sets. Thus, it is better to test models on multiple sets of the data. Fortunately, Scikit-Learn has many [built-in functions](http://scikit-learn.org/stable/modules/cross_validation.html) for sampling and resampling train/test sets:
* **K-fold:** divides all the samples in k groups of samples, called folds of equal sizes (if possible). The prediction function is learned using k - 1 folds, and the fold left out is used for test.
* **Leave One Out (LOO):** each learning set is created by taking all the samples except one, the test set being the sample left out. This cross-validation procedure does not waste much data as only one sample is removed from the training set.
* **Leave P Out (LPO):** very similar to LeaveOneOut as it creates all the possible training/test sets by removing p samples from the complete set.
* **Shuffle & Split:** generate a user defined number of independent train / test dataset splits. Samples are first shuffled and then split into a pair of train and test sets.
* **Stratified k-fold:** variation of k-fold which returns stratified folds: each set contains approximately the same percentage of samples of each target class as the complete set.
* **Stratified Shuffle & Split:**  variation of ShuffleSplit, which returns stratified splits, i.e which creates splits by preserving the same percentage for each target class as in the complete set.

We demonstrate two of these functions below.

In [ ]:
from sklearn.model_selection import StratifiedShuffleSplit
StratifiedShuffleSplit?

In [ ]:
sss = StratifiedShuffleSplit(n_splits=5, test_size=0.2, random_state=0)

for train_ix, test_ix in sss.split(X, y):
    print(train_ix.shape, test_ix.shape)
    
print(train_ix[:10])
print(test_ix[:10])

In [ ]:
## Define train sizes.
train_sizes = np.linspace(0.2,0.8,13)
n_train_sizes = train_sizes.shape[0]
n_splits = 10
test_size = 0.2

## Preallocate space for performance data.
performance = np.zeros((n_train_sizes,n_splits,3))

## Iteratively compute scores.
for i, train_size in enumerate(train_sizes):
    
    ## Initialize cross-validation tool.
    sss = StratifiedShuffleSplit(n_splits=n_splits, train_size=train_size, 
                                 test_size=test_size, random_state=0)
    
    for j, indices in enumerate(sss.split(X,y)):
        
        ## Extract indices.
        train_ix, test_ix = indices
        
        ## Fit SVC.
        fit = svc.fit(X[train_ix], y[train_ix])
        
        ## Compute performance scores.
        train_score = fit.score(X[train_ix], y[train_ix])
        test_score = fit.score(X[test_ix], y[test_ix])
        
        ## Store information.
        performance[i,j,0] = train_size
        performance[i,j,1] = train_score
        performance[i,j,2] = test_score

In [ ]:
## Convert to DataFrame.
performance = DataFrame(performance.reshape(n_train_sizes*n_splits,3), 
                        columns=('Train Size','Train', 'Test'))

## Melt into longlist.
performance = performance.melt(id_vars='Train Size', var_name='Model', value_name='Score')

## Plot.
sns.factorplot('Train Size', 'Score', 'Model', data=performance, aspect=3);

In [ ]:
from sklearn.model_selection import cross_val_score
cross_val_score?

In [ ]:
## Define parameters.
C_params = np.power( 10., [-2, -1, 0, 1, 2] )
n_c_params = C_params.shape[0]
kfold = 10

## Preallocate space for performance data.
performance = np.zeros((n_c_params,kfold,2))

for i, C in enumerate(C_params):
    
    ## Initialize SVC.
    svc = LinearSVC(penalty='l2', C=C, random_state=0)
    
    ## Perform cross-validation.
    scores = cross_val_score(svc, X, y, cv=kfold)
    
    ## Store.
    performance[i,:,0] = C
    performance[i,:,1] = scores

In [ ]:
## Convert to DataFrame.
performance = DataFrame(performance.reshape(n_c_params*kfold,2), columns=('C','Score'))

## Plot.
sns.swarmplot('C','Score',data=performance);